<a href="https://colab.research.google.com/github/Aleynaabulbul/covid19/blob/main/Covid19_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import os, sys
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.float_format', '{:.2f}'.format)
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Load data

dfcovid = pd.read_csv('/content/drive/MyDrive/aleyna/covid.csv')
dfvac = pd.read_csv('/content/drive/MyDrive/aleyna/country_vaccinations.csv')
dfpop = pd.read_csv('/content/drive/MyDrive/aleyna/pop.csv')
dfpop65up = pd.read_csv('/content/drive/MyDrive/aleyna/pop65up.csv')
dfgdp = pd.read_csv('/content/drive/MyDrive/aleyna/gdp.csv')

In [3]:
dfpop = dfpop[['Country Name','Country Code', '2018']]
dfpop = dfpop.rename(columns={'2018':'Population'})
merge1 = pd.merge(dfcovid, dfpop, left_on=['Country'], right_on = ['Country Name'], how='left')
dfgdp = dfgdp[['Country Name','2018']]
dfgdp = dfgdp.rename(columns={'2018':'GDP'})
merge1.loc[merge1['Country'] == 'US','Country'] = 'United States'
dfgdp.loc[dfgdp['Country Name'] == 'US','Country'] = 'Russia'
merge2 = pd.merge(merge1, dfgdp, left_on=['Country'], right_on = ['Country Name'], how='left')
merge2 = merge2.drop(columns = {'Country_y', 'Country Name_y'})
merge2 = merge2.rename(columns={'Country_x':'Country'})
dfvac = dfvac[['country','date','people_vaccinated']]
dfvac['people_vaccinated'] = dfvac.groupby('country')['people_vaccinated'].ffill()
merge3 = pd.merge(merge2, dfvac, left_on=['Date','Country'], right_on=['date','country'], how='left')
merge3 = merge3.drop(columns = {'country','date'})
merge3.loc[merge3['people_vaccinated'].isna()==True, 'people_vaccinated'] = 0
dfcovid_turkey = merge3[merge3['Country']== 'Turkey'].reset_index(drop=True)
test = dfcovid_turkey[dfcovid_turkey['Date'] >= '2021-03-01'].reset_index(drop=True)
train = merge3[merge3['Date'] < '2021-03-01'].reset_index(drop=True)
dataset = pd.concat([train,test]).reset_index(drop=True)
dataset2 = dataset.drop(columns = {'Date','Country','Country Code'})
dataset2['GDP'] = dataset2['GDP']/1000000000
dataset2.loc[dataset2['Population'].isna()==True,'Population'] = int(dataset2['Population'].mean())
dataset2.loc[dataset2['GDP'].isna()==True,'GDP'] = int(dataset2['GDP'].mean())

In [4]:
dataset2

,Confirmed,Recovered,Deaths,Country Name_x,Population,GDP,people_vaccinated
0,0,0,0,Afghanistan,37172386.00,19.36,0.00
1,0,0,0,Afghanistan,37172386.00,19.36,0.00
2,0,0,0,Afghanistan,37172386.00,19.36,0.00
3,0,0,0,Afghanistan,37172386.00,19.36,0.00
4,0,0,0,Afghanistan,37172386.00,19.36,0.00
...,...,...,...,...,...,...,...
77632,4929118,4554037,41527,Turkey,82319724.00,771.35,14132161.00
77633,4955594,4589501,41883,Turkey,82319724.00,771.35,14264014.00
77634,4977982,4626799,42187,Turkey,82319724.00,771.35,14391595.00
77635,4998089,4662328,42465,Turkey,82319724.00,771.35,14476881.00


In [5]:
# Turkey and Italy Model

turkeyitaly = dataset2[(dataset2['Country Name_x']== 'Turkey') | (dataset2['Country Name_x']== 'Italy') ].reset_index(drop=True)

X_test = turkeyitaly[-52:].drop(columns={'Confirmed', 'Country Name_x'}).reset_index(drop=True)
X_train = turkeyitaly[0:450].drop(columns={'Confirmed', 'Country Name_x'}).reset_index(drop=True)

y_train = turkeyitaly[0:450]['Confirmed'].to_numpy().reshape(-1,1)
y_test = turkeyitaly[-52:]['Confirmed'].to_numpy().reshape(-1,1)

#Model creation and predictions
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)

y_pred = regr.predict(X_test)

print('Coefficients: \n', regr.coef_)

# The mean squared error
print('Mean squared error: %.2f' % mean_squared_error(y_test, y_pred))

# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f' % r2_score(y_test, y_pred))

Coefficients: 
 [[ 1.48686354e+00  2.05039036e+00  1.99789104e-04 -1.19800762e-08
  -3.82360687e-01]]
Mean squared error: 8622323940619.66
Coefficient of determination: -16.95


In [6]:
#Turkey only model

turkey = dataset2[dataset2['Country Name_x'] == 'Turkey'].reset_index(drop=True)

X_test = turkey[-52:].drop(columns={'Confirmed', 'Country Name_x'}).reset_index(drop=True)
X_train = turkey[0:450].drop(columns={'Confirmed', 'Country Name_x'}).reset_index(drop=True)

y_train = turkey[0:450]['Confirmed'].to_numpy().reshape(-1,1)
y_test = turkey[-52:]['Confirmed'].to_numpy().reshape(-1,1)

from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)

y_pred = regr.predict(X_test)

print('Coefficients: \n', regr.coef_)

# The mean squared error
print('Mean squared error: %.2f' % mean_squared_error(y_test, y_pred))

# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f' % r2_score(y_test, y_pred))

Coefficients: 
 [[8.93710626e-01 1.59635999e+01 0.00000000e+00 1.53780911e-31
  1.17049970e-02]]
Mean squared error: 27909662814.87
Coefficient of determination: 0.94


In [7]:
y_pred = regr.predict(X_test)
>>> print('predicted response:', y_pred, sep='\n')

predicted response:
[[3028669.66465877]
 [3046441.54254895]
 [3064870.91645789]
 [3082260.87841585]
 [3101879.41961273]
 [3121779.74666535]
 [3140467.72194009]
 [3160473.21176469]
 [3181354.96536714]
 [3200901.92687928]
 [3218684.00684563]
 [3239042.2180269 ]
 [3262173.88017028]
 [3286202.24087062]
 [3310098.20633596]
 [3336034.52459342]
 [3361683.43289027]
 [3385693.67203048]
 [3414602.41467397]
 [3446854.42183998]
 [3486163.61709601]
 [3527299.17785777]
 [3565080.89249993]
 [3599153.6324598 ]
 [3630409.81434998]
 [3673483.49566168]
 [3727679.69412453]
 [3783621.81881682]
 [3839199.69263662]
 [3898182.54640573]
 [3950743.92725277]
 [3995374.2920427 ]
 [4045474.51572748]
 [4102626.67529112]
 [4156988.45427093]
 [4223673.0827652 ]
 [4284611.75353043]
 [4338797.19211969]
 [4390801.05548795]
 [4440712.98384116]
 [4488098.28161841]
 [4535454.79461984]
 [4581085.76781787]
 [4649082.14205566]
 [4728163.50535778]
 [4801347.40953279]
 [4840120.27986572]
 [4881960.90109192]
 [4920881.83525525]


In [ ]:
df=regr.score(X_test,y_test)
print(df)
dff=regr.score(X_train,y_train)
print(dff)

In [ ]:
X_train = dataset2[0:77568].drop(columns={'Confirmed'})
y_train = dataset2[0:77568]['Confirmed']
X_test = dataset2[77568:].drop(columns={'Confirmed'})
y_test = dataset2[77568:]['Confirmed']

y_train = y_train.to_numpy().reshape(-1,1)
y_test = y_test.to_numpy().reshape(-1,1)


In [ ]:
regr.predict(y_train)

In [ ]:
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)

y_pred = regr.predict(X_test)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
#Model creation and predictions
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)

y_pred = regr.predict(X_test)

print('Coefficients: \n', regr.coef_)

# The mean squared error
print('Mean squared error: %.2f' % mean_squared_error(y_test, y_pred))

# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f' % r2_score(y_test, y_pred))